In [2]:
# DIVIDE AND CONQUER
# This script divides all the reactions in the RMG model in half, switching half of them to NIST
# it then checks which has the more accurate flame speed and zooms in to try to find the reaction(s)
# which contributes the most to the flame speed accuracy

In [1]:
import cantera as ct
import cantera.ck2cti
import rmgpy.chemkin
import numpy as np
import subprocess

import csv

import mix_thermokinetics

import scipy
import copy
import os

In [2]:
# Load the models
# load David's 2-BTP model, minus the halogens
# https://github.com/comocheng/halogen_models/tree/main/combustion_symposium_paper/supporting_material

full_path = '/work/westgroup/nora/Code/projects/halogens/refrigerants/singles/Burgess_Comments/cantera/difftool/'


RMG_chemkin_path = full_path + 'no_halogens.inp'
RMG_dictionary_path = full_path + 'no_halogens_dict.txt'
RMG_transport_path = full_path + 'no_halogens_tran.dat'
RMG_cti_path = full_path + 'no_halogens.cti'

RMG_species_list, RMG_reaction_list = rmgpy.chemkin.load_chemkin_file(RMG_chemkin_path, dictionary_path=RMG_dictionary_path, transport_path=RMG_transport_path)
RMG_gas = ct.Solution(RMG_cti_path)


# Load NIST model without halogens
NIST_cti_path = full_path + 'NIST_no_halogens/no_halogens.cti'
NIST_chemkin_path = full_path + 'NIST_no_halogens/no_halogens.inp'
NIST_dictionary_path = full_path + 'NIST_no_halogens/no_halogens_dict.txt'
NIST_transport_path = full_path + 'NIST_no_halogens/no_halogens_tran.dat'

NIST_gas = ct.Solution(NIST_cti_path)
NIST_dict = rmgpy.chemkin.load_species_dictionary(NIST_dictionary_path)
NIST_species_list, NIST_reaction_list = rmgpy.chemkin.load_chemkin_file(NIST_chemkin_path, dictionary_path=NIST_dictionary_path, transport_path=NIST_transport_path)



**** WARNING ****
For species CH2OCH, discontinuity in cp/R detected at Tmid = 500
	Value computed using low-temperature polynomial:  8.39347
	Value computed using high-temperature polynomial: 9.1801


**** WARNING ****
For species CH2OCH, discontinuity in h/RT detected at Tmid = 500
	Value computed using low-temperature polynomial:  42.1991
	Value computed using high-temperature polynomial: 41.9615


**** WARNING ****
For species CH2OCH, discontinuity in s/R detected at Tmid = 500
	Value computed using low-temperature polynomial:  33.7069
	Value computed using high-temperature polynomial: 33.5121


**** WARNING ****
For species C4H5-2, discontinuity in h/RT detected at Tmid = 1000
	Value computed using low-temperature polynomial:  47.6524
	Value computed using high-temperature polynomial: 48.4362


**** WARNING ****
For species C4H5-2, discontinuity in s/R detected at Tmid = 1000
	Value computed using low-temperature polynomial:  52.4292
	Value computed using high-temperature polyno

In [3]:
# pick a random order for dividing the reactions
N = len(RMG_reaction_list)
np.random.seed(379)
order = np.random.permutation(N)
order

array([646, 295, 466, 251,  42,  80, 265, 108, 573, 311, 398,  21, 341,
       286,  41, 218, 540,  94, 727, 595, 247, 245, 617,  43, 121, 713,
       138, 761, 267, 550, 641, 294, 176, 567, 764, 785, 397, 659, 289,
        61, 436, 706, 434, 423, 430, 565, 291,  57, 359, 545,  64, 668,
       399,   1, 734, 809, 640, 367, 506, 559, 135,  13,  96, 755, 183,
        84,  74, 385, 622, 339, 607, 337, 240,  54, 390, 333, 470,  99,
       623, 788, 150, 422, 403, 437, 279, 495, 318, 558,  55, 274, 457,
       476,  87, 200, 621, 142, 130, 642, 799, 660, 592, 598, 577, 670,
       628, 736, 169, 264, 630, 344, 544, 298, 652, 539, 791, 512, 167,
       687, 732, 112, 325, 198, 366, 276, 119, 822,  39,  59, 177,  60,
       719, 101, 688, 665, 779, 194, 378, 730,  47, 717, 552, 118, 500,
       675, 228, 777, 211, 498, 373, 612, 790, 182, 174,  92, 215, 346,
       738, 283,  20, 682, 278,  69, 760, 350, 418, 709, 411,  32, 224,
       296, 244, 508, 424,  52, 576, 331, 520,  58, 574, 572, 75

In [4]:
# get the mapping between RMG and NIST models
# Species Diff
common_species = []
RMG2NIST_mapping = {}
NIST2RMG_mapping = {}
for i, rmg_sp in enumerate(RMG_species_list):
    for j, nist_sp in enumerate(NIST_species_list):
        if rmg_sp.is_isomorphic(nist_sp):
            RMG2NIST_mapping[i] = j
            NIST2RMG_mapping[j] = i
            common_species.append([rmg_sp, nist_sp])
            break

# Reaction Diff
common_reactions = []
RMG2NIST_rxn_mapping = {}
NIST2RMG_rxn_mapping = {}
for i, rmg_rxn in enumerate(RMG_reaction_list):
    for j, nist_rxn in enumerate(NIST_reaction_list):
        if rmg_rxn.is_isomorphic(nist_rxn):
            RMG2NIST_rxn_mapping[i] = j
            NIST2RMG_rxn_mapping[j] = i
            common_reactions.append([rmg_rxn, nist_rxn])
            break
print(f'{len(common_species)} common species')
print(f'{len(common_reactions)} common reactions')

45 common species
240 common reactions


In [5]:
# Convert the NIST species in the reactions to RMG species, but keep the NIST kinetics
def NIST2RMG(nist_reaction):
    rmg_reaction = copy.deepcopy(nist_reaction)
    reactants = []
    for reactant in nist_reaction.reactants:
        try:
            NIST_species_index = NIST_species_list.index(reactant)
            reactants.append(RMG_species_list[NIST2RMG_mapping[NIST_species_index]])
        except ValueError:
            if reactant in RMG_species_list:
                reactants.append(reactant)
        
    rmg_reaction.reactants = reactants
    
    products = []
    for product in nist_reaction.products:
        try:
            NIST_species_index = NIST_species_list.index(product)
            products.append(RMG_species_list[NIST2RMG_mapping[NIST_species_index]])
        except ValueError:
            if product in RMG_species_list:
                products.append(product)
    rmg_reaction.products = products
    
    return rmg_reaction

In [6]:
def RMG2NIST(RMG_reaction):
    # takes in the RMG_reaction object to convert
    RMG_index = RMG_reaction_list.index(RMG_reaction)
    if RMG_index not in RMG2NIST_rxn_mapping.keys():
        # this reaction does not exist in NIST, so it will be deleted. return None
        return
    NIST_index = RMG2NIST_rxn_mapping[RMG_index]
    NIST_reaction = NIST_reaction_list[NIST_index]
    
    # convert the NIST model species in the NIST_reaction to RMG model species
    return NIST2RMG(NIST_reaction)

In [7]:
divide_dir = 'divide'
os.makedirs(divide_dir, exist_ok=True)

In [82]:
first_half = False

In [83]:
# Round 0 - firsthalf-0.24669158902982635, secondhalf-0.3508091952479528
num_reactions_0 = int(N / 2)
# if first_half:
#     reactions_to_use_NIST = order[0: num_reactions_0]
# else:
#     reactions_to_use_NIST = order[num_reactions_0:]

# # Round 1  first half - 0.31077500873861313  secondhalf - 0.2494298527669706
num_reactions_1 = int(N / 4)
# if first_half:
#     reactions_to_use_NIST = order[num_reactions_0: num_reactions_0 + num_reactions_1]
    
# else:
#     reactions_to_use_NIST = order[num_reactions_0 + num_reactions_1: num_reactions_0 + 2 * num_reactions_1]
    
# Round 2 first half - 0.2975234946950403 secondhalf - 0.2650320277534765
# num_reactions_2 = int(N / 8)
# if first_half:
#     reactions_to_use_NIST = order[num_reactions_0: num_reactions_0 + num_reactions_2]
# else:
#     reactions_to_use_NIST = order[num_reactions_0 + num_reactions_2:]




# convert the indicated reactions to use the NIST kinetics
new_reaction_list = []
deleted_duplicates = []
for i in range(0, len(RMG_reaction_list)):
    if i in reactions_to_use_NIST:
        new_reaction = RMG2NIST(RMG_reaction_list[i])
        if new_reaction:
            new_reaction_list.append(new_reaction)
        elif RMG_reaction_list[i].duplicate:
            deleted_duplicates.append(RMG_reaction_list[i])
    else:
        new_reaction_list.append(RMG_reaction_list[i])

In [84]:
# get rid of duplicates
for i, rxn in enumerate(new_reaction_list):
    if rxn.duplicate:
        duplicate_still_exists = False
        for j, rxn2 in enumerate(new_reaction_list):
            if rxn.is_isomorphic(rxn2) and rxn != rxn2:
                duplicate_still_exists = True
                break
        if not duplicate_still_exists:
            rxn.duplicate = False
            
# mark reactions that are duplicates
for i, rxn in enumerate(new_reaction_list):
    if not rxn.duplicate:
        duplicate = False
        for j, rxn2 in enumerate(new_reaction_list):
            if rxn.is_isomorphic(rxn2) and rxn != rxn2:
                duplicate = True
                break
        if duplicate:
            rxn.duplicate = True

In [85]:
# save the models
if first_half:
    chemkin_file = os.path.join(divide_dir, 'chem_0.inp')
else:
    chemkin_file = os.path.join(divide_dir, 'chem_1.inp')
rmgpy.chemkin.save_chemkin_file(chemkin_file, RMG_species_list, new_reaction_list, verbose=True, check_for_duplicates=True)

# if first_half:
#     subprocess.run(['ck2cti', f'--input={chemkin_file}', f'--transport={RMG_transport_path}', f'--output={divide_dir}/chem_0.cti'])
# else:
#     subprocess.run(['ck2cti', f'--input={chemkin_file}', f'--transport={RMG_transport_path}', f'--output={divide_dir}/chem_1.cti'])

'/work/westgroup/nora/Code/projects/halogens/refrigerants/singles/Burgess_Comments/cantera/difftool/no_halogens_tran.dat'

In [87]:
path_to_round_2 = '/work/westgroup/nora/Code/projects/halogens/refrigerants/singles/Burgess_Comments/divide_and_conquer/divide/round_2/'
chem_0 = path_to_round_2 + 'chem_0.cti'
chem_1 = path_to_round_2 + 'chem_1.cti' 

# RUN FLAME SPEED


In [88]:
#######Nora's flame speed script instead#####################
#########calculates flamespeeds for the .cti files at different equivalence ratios. Uses initial guess from the previous model #############################

import cantera as ct
import numpy as np
import pandas as pd
import os
import csv 
print("Running Cantera Version: " + str(ct.__version__))

To = 298
Po = ct.one_atm

gas = ct.Solution(chem_0)


#vol_frac_list = np.arange(0.5, 1.2, step=0.07)


#can use below to only test one vol_frac
vol_frac_list = [0.095]


results = {}

for i in  range(len(vol_frac_list)):
    try: 
        
        tol_ss = [1.0e-13, 1.0e-9]  #abs and rel tolerances for steady state problem
        tol_ts = [1.0e-13, 1.0e-9]  #abs and rel tie tolernces for time step function
        
        x = vol_frac_list[i]
        norm_ox = (1-x)*.21
        
        
        print(f'****************************starting new volume fraction: {x}**************************')

        vol_frac_dict = {'CH4(3)': (x/norm_ox), 'O2(4)':((1-x)*.21)/norm_ox, 'N2':((1-x)*0.79)/norm_ox}
        print(vol_frac_dict)
        print(f"O2/CH4 ratio = {vol_frac_dict['O2(4)']/vol_frac_dict['CH4(3)']}. Complete combustion takes 2")
        gas.TPX = To, Po, vol_frac_dict
        width = 0.08
        flame = ct.FreeFlame(gas, width=width)
        flame.flame.set_steady_tolerances(default=tol_ss)   #set tolerances
        flame.flame.set_transient_tolerances(default=tol_ts)
        #flame.set_refine_criteria(ratio=3, slope=0.1, curve=0.1) 
        flame.set_refine_criteria(ratio=5, slope=0.25, curve=0.27)
        flame.max_time_step_count = 900
        loglevel = 1 

        flame.solve(loglevel=loglevel, auto=True)
        Su = flame.u[0]
        results[x] = Su
        sltn = flame.to_solution_array()
        df1 = sltn.to_pandas()
        #edited this here!! index=False
        df1.to_csv(f'./data/David_test_{x}.csv', index=False)
    except Exception as e: 
        print(f'********************passed volume fraction:{vol_frac_list[i]}, error: {e}*************************************')
        pass


vol_fracs = list(results.keys())
flame_speeds = list(results.values())


print("volume fractions are:")
print(vol_fracs)

print("flame speeds are:")
print(flame_speeds)


with open('final_calcs_David.csv', 'w+') as g:
    writers = csv.writer(g)
    writers.writerow(vol_fracs)
    writers.writerow(flame_speeds)

        



Running Cantera Version: 2.4.0
****************************starting new volume fraction: 0.095**************************
{'CH4(3)': 0.4998684556695607, 'O2(4)': 1.0, 'N2': 3.7619047619047623}
O2/CH4 ratio = 2.0005263157894735. Complete combustion takes 2

************ Solving on 8 point grid with energy equation enabled ************

..............................................................................
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     0.0001709      3.832
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps      0.004379      3.736
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     4.871e-05      5.459
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps       1.3e-05      6.144
Attempt Newton solution of steady-state problem...    success.

Problem solved on [9] point grid(s).

.......................................................

Attempt Newton solution of steady-state problem...    success.

Problem solved on [58] point grid(s).

..............................................................................
##############################################################################
Refining grid in flame.
    New points inserted after grid points 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 
    to resolve C(T)(17) C2H(23) C2H2(24) C2H3(30) C2H4(31) C2H5(33) C2H6(32) C3H4(2596) C3H4(5216) CH(18) CH2(S)(26) CH2(T)(19) CH2CHO(36) CH2CO(29) CH2O2(67) CH2OH(34) CH3(20) CH3CHO(37) CH3O(28) CH3O2(73) CH3OO(201) CHCHOH(59) CHO3(90) H2CC(25) H2O2(13) HCCO(22) HCO(16) S(105) S(26591) 
##############################################################################

..............................................................................
Attempt Newton solution of steady-state problem...    success.

Problem solved on [75] point grid(s).

.....................................................................

0.24669158902982635